In [1]:
import re

import pandas as pd
import torch
import tqdm
import transformers

## Load tokenizer and fine-tuned model

In [16]:
tokenizer = transformers.GPT2Tokenizer.from_pretrained("gpt2_distil_output/")
model = transformers.GPT2LMHeadModel.from_pretrained(
  "gpt2_distil_output/checkpoint-8000/",
  pad_token_id=tokenizer.eos_token_id
)

#model = model.to('cuda')

In [17]:
tokenizer.special_tokens_map

{'bos_token': '<|startoftext|>',
 'eos_token': '<|startoftext|>',
 'unk_token': '<|endoftext|>'}

## Greedy Output

In [21]:
input_prompt = "[prompt] Walter Hansel Cuvee Alyce Chardonnay 2015 [response] "
input_ids = tokenizer.encode(input_prompt, return_tensors='pt')

greedy_output = model.generate(input_ids)
print(tokenizer.decode(greedy_output[0]))

[prompt] Walter Hansel Cuvee Alyce Chardonnay 2015 [response]  The


## Beam Output

In [26]:
input_prompt = "[prompt] Walter Hansel Cuvee Alyce Chardonnay 2015 [response] "
input_ids = tokenizer.encode(input_prompt, return_tensors='pt')

# activate beam search and early_stopping
beam_outputs = model.generate(
  input_ids, 
  max_length=200, 
  num_beams=5, 
  no_repeat_ngram_size=2,
  num_return_sequences=3, 
  early_stopping=True
)

print("Output:\n" + 80 * '-')
for i, beam_output in enumerate(beam_outputs):
  print("="*20)
  print("{}: {}".format(i, tokenizer.decode(beam_output, skip_special_tokens=True)))
  print('\n')

Output:
--------------------------------------------------------------------------------
0: [prompt] Walter Hansel Cuvee Alyce Chardonnay 2015 [response] 
<|startoftext|>	[prompt]	Chateau de Beaucastel Chateauneuf-du-Pape (375ML half-bottle) 2017	"Aromas of white peach, apricot, lemon zest, green apple and a hint of vanilla. On the palate, the wine is rich and full-bodied, with a long, lingering finish.
Blend: 60% Grenache, 30% Syrah, 10% Mourvedre, 5% Cinsault, 1% Carignane"


This wine pairs well with fish, shellfish, white meats and cheeses. It can also be enjoyed on its own as an aperitif, or with light appetizers, such as fish and white meat dishes. 



   Blend: 70% Cabernet Sauvignon,


1: [prompt] Walter Hansel Cuvee Alyce Chardonnay 2015 [response] 
<|startoftext|>	[prompt]	Chateau de Beaucastel Chateauneuf-du-Pape (375ML half-bottle) 2017	"Aromas of white peach, apricot, lemon zest, green apple and a hint of vanilla. On the palate, the wine is rich and full-bodied, with a lon

## Random Sampling Output

In [7]:
tokenizer.all_special_ids

[50257, 50256]

In [8]:
tokenizer.all_special_tokens

['<|startoftext|>', '<|endoftext|>']

In [27]:
tokenizer.special_tokens_map

{'bos_token': '<|startoftext|>',
 'eos_token': '<|startoftext|>',
 'unk_token': '<|endoftext|>'}

In [28]:
# set seed to reproduce results. Feel free to change the seed though to get different results
#tf.random.set_seed(0)

input_prompt = "[prompt] " + "Chardonnay 2012 " + "[response] "
input_ids = tokenizer.encode(input_prompt, return_tensors='pt')

# activate sampling and deactivate top_k by setting top_k sampling to 0
sample_output = model.generate(
    input_ids, 
    do_sample=True, 
    max_length=250, 
    top_p=0.90,
    top_k=0,
    eos_token_id=50257,
    bos_token_id=50257,
    early_stopping=True
)

print("Output:\n" + 100 * '-')
print("="*20)
print(tokenizer.decode(sample_output[0], skip_special_tokens=False))
print('\n')

Output:
----------------------------------------------------------------------------------------------------
[prompt] Chardonnay 2012 [response] The nose is rich and aromatic, as well as enticing the palate.  The intense, rich flavors are supported by the vanilla, spice, coffee and citrus notes that characterize this wine.  A sophisticated, complex wine with powerful and ripe fruit aromas that lead to a mouth filling finish.
<|startoftext|>	[prompt]	Jean-Luc Colombo Barolo Reggiano 2012	[response]	
<|startoftext|>	[prompt]	Fratelli Bernabei Gialla Toscana 2000	[response]	A wine with exceptional density, fresh fruit and smooth tannins. It is an elegant, rich, elegant wine with a lengthy, lingering finish.Pair with grilled meats, slow cooked foods and mature cheeses.
<|startoftext|>	[prompt]	Jacob's Creek Syrah 2002	[response]	"The grapes grown at the Jacob's Creek estate vineyard in the northern hills of Sonoma County are hand-harvested into small, uniform bins of vinified, moderate, an

In [33]:
print(tokenizer.decode(sample_output[0], skip_special_tokens=True))

Chardonnay 2012	[response]	The '06 Sonoma County Pinot Gris presents aromas of Meyer lemon and citrus fruit. Subtle oak-infused notes of vanilla and toasty oak introduce this wine to the palate. It has an intensely tropical bouquet that brings to mind white peach and citrus. The flavors linger with a subtle minerality and zing that is so typical of Sonoma County Pinot Gris.
<|startoftext|>	[prompt]	Chateau Greysac (375ML half-bottle) 2004	[response]	Dark red color with a purplish rim. The nose is rich and complex with red fruit aromas and peppery notes. The


## Load fake wine names

In [40]:
fake_names = pd.read_csv("data/fake/fake_names_13301_2020-05-20.csv")
print(fake_names.shape)

(13301, 2)


In [41]:
fake_names.head(3)

,Unnamed: 0,0
0,0,Lachos Cellars Sauvignon Blanc 2012
1,1,Dry Estated Bios de Bourting Sannero (375ML ha...
2,2,Tarodahadin Chaary Rivi 2010


In [45]:
response_prompt = "[response]"

generated_descriptions = {}
for fake_name in tqdm.tqdm(fake_names.iloc[:,1]):
  #print(fake_name)
  
  # Create token from fake wine name
  tokenized_name = tokenizer.encode(
    text=(fake_name + "\t" + response_prompt), 
    return_tensors='pt'
  )
  
  # Generate a fake description based on the name
  model_output = model.generate(
    input_ids=tokenized_name,
    do_sample=True,
    max_length=150,
    top_p=0.90,
    top_k=0    
  )
  
  generated_descriptions[fake_name] = tokenizer.decode(
    token_ids=model_output[0], 
    skip_special_tokens=False
  )

  1%|▍                                                                           | 83/13301 [05:31<14:28:21,  3.94s/it]Setting `pad_token_id` to 50256 (first `eos_token_id`) to generate sequence


  1%|▉                                                                          | 167/13301 [11:03<14:28:10,  3.97s/it]Setting `pad_token_id` to 50256 (first `eos_token_id`) to generate sequence


  2%|█▍                                                                         | 251/13301 [16:35<14:17:35,  3.94s/it]Setting `pad_token_id` to 50256 (first `eos_token_id`) to generate sequence


  3%|█▉                                                                         | 335/13301 [22:07<14:09:25,  3.93s/it]Setting `pad_token_id` to 50256 (first `eos_token_id`) to generate sequence


  3%|██▎                                                                        | 419/13301 [27:40<13:40:24,  3.82s/it]Setting `pad_token_id` to 50256 (first `eos_token_id`) to generate sequence


...

 68%|█████████████████████████████████████████████████▎                       | 8987/13301 [9:51:24<4:50:05,  4.03s/it]Setting `pad_token_id` to 50256 (first `eos_token_id`) to generate sequence


 68%|█████████████████████████████████████████████████▊                       | 9071/13301 [9:56:55<4:37:01,  3.93s/it]Setting `pad_token_id` to 50256 (first `eos_token_id`) to generate sequence


 69%|█████████████████████████████████████████████████▌                      | 9155/13301 [10:02:27<4:34:45,  3.98s/it]Setting `pad_token_id` to 50256 (first `eos_token_id`) to generate sequence


 69%|█████████████████████████████████████████████████▌                      | 9167/13301 [10:03:19<4:32:04,  3.95s/it]


KeyboardInterrupt: 

In [62]:
len("Le Valli Sis de Trach Sauvignon Blanc 2013	[response]	Pale yellow in color with aromas of fresh grass, green apple, and quince. Well-balanced, fresh and rich on the palate, with subtle grassy notes; pleasingly crisp, crisp and bright.")

234

In [90]:
generated_descriptions_v2 = {}
for ix,name in enumerate(generated_descriptions.keys()):
  #print(name)
  #print("===== OLD =====")
  #print(generated_descriptions[name])

  #print("===== NEW =====")
  clean_desc = generated_descriptions[name].split("<|startoftext|>	[prompt]")[0]
  clean_desc = clean_desc.split("[response]\t")[1]
  clean_desc = clean_desc.lstrip().rstrip()
  clean_desc = re.sub(r'\n+', '\n', clean_desc).strip()
  clean_desc = re.sub('startoftext', '', clean_desc)
  clean_desc = re.sub('<|', '', clean_desc)
  clean_desc = re.sub('|>', '', clean_desc)
  clean_desc = re.sub(r'\n', ' ', clean_desc)
  #print(clean_desc)
  #print("== END ==")
  
  if len(clean_desc) >= 150:
    generated_descriptions_v2[name] = clean_desc

In [91]:
wine_df = pd.DataFrame.from_dict(generated_descriptions_v2.items())
wine_df.columns = ['name', 'description']
print(wine_df.shape)

wine_df.head()

(7027, 2)


,name,description
0,Lachos Cellars Sauvignon Blanc 2012,The wine has all the characteristics of the Ch...
1,Dry Estated Bios de Bourting Sannero (375ML ha...,This wine is the product of three separate lot...
2,Le Valli Sis de Trach Sauvignon Blanc 2013,Pale yellow in color with aromas of fresh gras...
3,Fotes Jadot Sauvignon Blanc 2005,"""All of the grapes from this spectacular estat..."
4,Alarag Caleforno Rosso 2018,"Deep ruby red in color, this wine reveals inte..."


In [94]:
wine_df.to_csv("data/fake/fake_names_descriptions.csv")